In [1]:
!nvidia-smi

Sat May  6 13:48:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation

In [3]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached langchain-0.0.160-py3-none-any.whl (756 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached frozenlist-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (149 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)
  Using cached typing_inspect-0.8.0-py3-none-any.whl (8.7 kB)
  Using cached marshmallow-3.19.0-py3-none-any.whl (49 kB)
  Using cached marshmallow_enum-1.5.1-py2

In [4]:
import os
os.environ["OPENAI_API_KEY"]="..."

## LLMs: language model로부터 prediction들을 얻자.
가장 기본적인 LangChain의 building block은 input에 대해 LLM을 불러 내는 것이다. 해당 코드의 목적은 회사가 무슨 일을 하는가에 따라 회사명을 생성해 보는 것이다.

In [6]:
from langchain.llms import OpenAI

# Random성을 가중시키기 위해 temperature를 높게 설정해본다.
llm = OpenAI(temperature=0.9)

## KO
text = "다채로운 양말을 만드는 회사에 어울리는 회사명은 무엇일까?"
print(llm(text))

## EN
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



★열정같이 따뜻해지는 양말★


Fuzzy Feet Toesies.


## Prompt Templates: LLMs를 위한 prompt들을 관리한다.
일반적으로 LLM이 어플리케이션에서 사용될 떄, 유저의 input을 그대로 LLM에 넣지는 않을 것이다.
대신에, 아마도 당신은 유저의 input을 받고 prompt를 구성한 다음 LLM에게 그것을 전달해 줄 것이다.

위 예시를 업데이트 해본다면, 우리는 유저에게 회사가 하는 일만을 입력 받고 그 정보들로 prompt의 format을 맞춰주는 것을 생각해 볼 수 있겠다. 이 작업은 LangChain을 통해서면 매우 쉽게 진행이 가능하다!

In [7]:
from langchain.prompts import PromptTemplate

## KO
ko_prompt = PromptTemplate(
    input_variables=["product"],
    template="{product}를(을) 만드는 회사에 어울리는 좋은 이름이 뭘까?"
)
print(ko_prompt.format(product="다채로운 양말"))

## EN
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="colorful socks"))

다채로운 양말를(을) 만드는 회사에 어울리는 좋은 이름이 뭘까?
What is a good name for a company that makes colorful socks?


## Chains: LLMs와 prompt들을 다중 스텝 워크플로우(multi-step workflow)에서 합쳐보자.
지금까지는 PromptTemplate와 LLM을 단독으로 사용했었다. 하지만 실제 어플리케이션에서는 하나씩 단독으로 사용되는것이 아나라 이들의 조합으로 활용될 것이다.

LangChain의 chain은 각각의 LLMs부터 또 다른 chain까지의 link들로 구성된다. chain의 가장 코어 타입은 LLMChain으로 PromptTemplate와 LLM으로 구성되어있다. 앞의 예시를 더욱 확징해 보면, user input을 받아 PromptTemplate을 통해 format시키고, 이를 LLM에 넣어주는 동작이 LLMChain 내부에서 일어날 것이다.

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

## EN
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("colorfulk socks"))

## KO
ko_chain = LLMChain(llm=llm, prompt=ko_prompt)
print(ko_chain.run("다채로운 양말"))



Happy Socks & Co.


1. 컬러풀한 양말 스타일
2. 따뜻한 양말 특강
3. 양말 스트릭트
4. 다채로운 스타일 피트
5. 신뢰할 수 있는 양말 세트
6. 꾸며진 양말 전문가
7. 누구나 소중하게 어울리는 양말 클라우드
8. 지속적인 양말 디자인
9. 따뜻
